Strategy:

We are given certain campaigns and certain days.
Each campaign has a specified budget for each day, and for each hour in any day for that campaign there are keywords with a given cost and certain profit that can be generated from these keywords.
We have to train a random forest that looks at data and is able to predict for each campaign each day each hour should we bid for each keyword possible or not.
This random forest is our actor since here we make predictions.

We use reinforcement learning on top of a random forest regressor to do this.
Before training the forest WE SHOULD ALREADY KNOW WETHER OR NOT TO PLACE BID ON EACH KEYWORD.
To do this we use knapsack algorithm.
The knapsack algorithm is our critic since this is what rewards the actions of the actor.

So basically we run a knapsack algorithm for all the training data and get the best bids for each row,
then our model makes a prediction for the bid and we compare the bid with the best bid. 

This training and testing process takes place in a parallel fashion.
We do not need to test or train in a sequential manner.



In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import sys
sys.path.append(<FOLDER PATH>)
from environment import env
from util import Util

/content/gdrive/MyDrive/freelancer-narender-s-261121/qlearning FINAL 5


In [3]:
!pip3 install pickle5

     |████████████████████████████████| 256 kB 5.4 MB/s 


In [4]:
import pickle5 
import numpy as np
import pandas as pd
import tensorflow as tf

In [5]:
with open(<DATASET PATH>, 'rb') as f:
  data = pickle5.load(f)

In [7]:
#we will only keep these columns in the dataframe others we will not use
columnsWeNeed=[
        'keywordId',
        'campaignId',
        'tAcos',
        'price',
        'hour',
        'month',
        'day',
        'dayVal',
        'cost',
        'sales',
        'campaignBudget'
      ]

In [8]:
data=data.dropna()#remove all rows with na values
Util.fillDates(data)#use the util function to fill up the date columns we need 
data=Util.convertToCat(data,['campaignId','keywordId'])#convert these two into integers since they are very large values and a very few unique values

In [9]:
data=data[columnsWeNeed]#extract only the columns we need

In [10]:
# #the code below is used to standardize the given columns. Standardization is the formula written at the end
# stdzColumns=[
#         'keywordId',
#         'tAcos',
#         'hour',
#         'month',
#         'day',
#       ]
# data[stdzColumns]=(data[stdzColumns]-data[stdzColumns].mean())/data[stdzColumns].std()

comment above cell if we dont want standardization

In [11]:
data=data[data['cost']>=0]#use only rows where cost is greater than/equal to zero

In [12]:
dataZero=data[(data['sales']==0)|(data['price']==0)].sample(4000)
# dataZero=data[(data['sales']==0)|(data['price']==0)]#this is the data with zero profit possible
dataPos=data[(data['sales']>0)&(data['price']>0)]#this is the data with positive profit possible

In [15]:
from sklearn.model_selection import train_test_split
with tf.device("/device:GPU:0"):
  trainPos, testPos = train_test_split(dataPos, test_size=0.2)#split the positive profit data into train and test
  trainZero, testZero = train_test_split(dataZero, test_size=0.2)#split the zero profit data into train and test
  train=pd.concat([trainPos,trainZero])#concat the zero and +ve profit data
  test=pd.concat([testPos,testZero])
  train = train.sample(frac=1)#shuffle the training data
  test = test.sample(frac=1)#shuffle the testing data

In [17]:
with tf.device("/device:GPU:0"):
  n_estimators=400
  stdz=False
  usePca=False
  pcac=7
  columns=['campaignBudget','keywordId','campaignId','tAcos','price','hour','month','day']
  #instantiate the environment with above parameters they are all explained in environment.py
  Env=env(n_estimators=n_estimators,columns=columns,usePca=usePca,pcac=pcac)

In [18]:
with tf.device("/device:GPU:0"):
  #train the environment
  #look at environment to completely understand 
  Env.train(data=train,stdz=stdz)

0    0
0    100
0    200
0    300
1    0
1    100
1    200
1    300
2    0
2    100
2    200
2    300
3    0
3    100
3    200
3    300
4    0
4    100
4    200
4    300
5    0
5    100
5    200
5    300
6    0
6    100
6    200
6    300
7    0
7    100
7    200
7    300
8    0
8    100
8    200
8    300
9    0
9    100
9    200
9    300
10    0
10    100
10    200
10    300
11    0
11    100
11    200
11    300
12    0
12    100
12    200
12    300


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 400
building tree 2 of 400
building tree 3 of 400
building tree 4 of 400
building tree 5 of 400
building tree 6 of 400
building tree 7 of 400
building tree 8 of 400
building tree 9 of 400
building tree 10 of 400
building tree 11 of 400
building tree 12 of 400
building tree 13 of 400
building tree 14 of 400
building tree 15 of 400
building tree 16 of 400
building tree 17 of 400
building tree 18 of 400
building tree 19 of 400
building tree 20 of 400building tree 21 of 400

building tree 22 of 400building tree 23 of 400

building tree 24 of 400
building tree 25 of 400
building tree 26 of 400building tree 27 of 400

building tree 28 of 400
building tree 29 of 400
building tree 30 of 400
building tree 31 of 400
building tree 32 of 400
building tree 33 of 400
building tree 34 of 400
building tree 35 of 400
building tree 36 of 400
building tree 37 of 400
building tree 38 of 400
building tree 39 of 400
building tree 40 of 400
building tree 41 of 400
building tree 42 of 400
b

[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    0.3s


building tree 53 of 400
building tree 54 of 400
building tree 55 of 400
building tree 56 of 400
building tree 57 of 400
building tree 58 of 400
building tree 59 of 400
building tree 60 of 400
building tree 61 of 400
building tree 62 of 400
building tree 63 of 400
building tree 64 of 400
building tree 65 of 400
building tree 66 of 400
building tree 67 of 400
building tree 68 of 400
building tree 69 of 400
building tree 70 of 400
building tree 71 of 400building tree 72 of 400

building tree 73 of 400building tree 74 of 400

building tree 75 of 400building tree 76 of 400

building tree 77 of 400
building tree 78 of 400
building tree 79 of 400
building tree 80 of 400
building tree 81 of 400
building tree 82 of 400
building tree 83 of 400
building tree 84 of 400
building tree 85 of 400
building tree 86 of 400
building tree 87 of 400
building tree 88 of 400
building tree 89 of 400building tree 90 of 400

building tree 91 of 400
building tree 92 of 400
building tree 93 of 400
building tree 94

[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:    1.3s


building tree 183 of 400
building tree 184 of 400
building tree 185 of 400
building tree 186 of 400
building tree 187 of 400
building tree 188 of 400
building tree 189 of 400
building tree 190 of 400
building tree 191 of 400building tree 192 of 400

building tree 193 of 400
building tree 194 of 400
building tree 195 of 400
building tree 196 of 400
building tree 197 of 400building tree 198 of 400

building tree 199 of 400
building tree 200 of 400
building tree 201 of 400building tree 202 of 400

building tree 203 of 400building tree 204 of 400

building tree 205 of 400building tree 206 of 400

building tree 207 of 400
building tree 208 of 400
building tree 209 of 400
building tree 210 of 400
building tree 211 of 400
building tree 212 of 400
building tree 213 of 400
building tree 214 of 400
building tree 215 of 400
building tree 216 of 400
building tree 217 of 400
building tree 218 of 400
building tree 219 of 400
building tree 220 of 400
building tree 221 of 400
building tree 222 of 400


[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed:    2.9s


building tree 367 of 400building tree 368 of 400

building tree 369 of 400building tree 370 of 400

building tree 371 of 400
building tree 372 of 400
building tree 373 of 400
building tree 374 of 400
building tree 375 of 400
building tree 376 of 400
building tree 377 of 400building tree 378 of 400

building tree 379 of 400
building tree 380 of 400
building tree 381 of 400
building tree 382 of 400
building tree 383 of 400
building tree 384 of 400
building tree 385 of 400
building tree 386 of 400
building tree 387 of 400
building tree 388 of 400
building tree 389 of 400
building tree 390 of 400
building tree 391 of 400
building tree 392 of 400
building tree 393 of 400
building tree 394 of 400
building tree 395 of 400
building tree 396 of 400
building tree 397 of 400
building tree 398 of 400
building tree 399 of 400
building tree 400 of 400


[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:    3.2s finished


In [19]:
with tf.device("/device:GPU:0"):
  #get predictions but also get metrices
  res=Env.test(data=test,showPer=True,stdz=stdz)

0    0
0    100
0    200
0    300
1    0
1    100
1    200
1    300
2    0
2    100
2    200
2    300
3    0
3    100
3    200
3    300
4    0
4    100
4    200
4    300
5    0
5    100
5    200
5    300
6    0
6    100
6    200
6    300
7    0
7    100
7    200
7    300
8    0
8    100
8    200
8    300
9    0
9    100
9    200
9    300
10    0
10    100
10    200
10    300
11    0
11    100
11    200
11    300
12    0
12    100
12    200
12    300


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 158 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 361 tasks      | elapsed:    0.2s


confusion_matrix:
[[ 565  235]
 [ 180 1361]]
precision_score:
0.8527568922305765
recall_score:
0.8831927319922128
accuracy_score:
0.8227253310551047
f1_score:
0.8677080012751037


[Parallel(n_jobs=2)]: Done 400 out of 400 | elapsed:    0.2s finished


In [21]:
#THE RESULTS CAN BE IMPROVED BY HYPERPARAMETER TUNING